# Tensor Product Experiment

In [1]:
import torch
from einops import einsum

## General Gate Product

This handles the product of two general quantum gates, which computationally is tensor contractions.

### Example

![general_gate_product](./images/tensor_network_experiment_circuit.png)

In [2]:
# |export tensor_utils
from einops import einsum
from typing import List, Set


def tensor_contract(*tensors, ein_expr: str, dims: List[Set[str]] | Set[str]) -> torch.Tensor:
    assert len(tensors) >= 2, "At least two tensors are needed for contraction"
    if isinstance(dims, Set):
        dims = [dims]
    elif isinstance(dims, List):
        pass
    else:
        raise ValueError(f"Invalid type for dims: {type(dims)}")

    for dim_set in dims:
        for d in dim_set:
            assert isinstance(d, str), f"Dimension name in a set must be a string, got {type(d)}"
            assert d in ein_expr, f"Dimension {d} not found in ein_expr"

    total_dims_num = sum(len(d) for d in dims)
    all_dims = set()
    for dim_set in dims:
        all_dims.update(dim_set)

    assert len(all_dims) == total_dims_num, "Sets of dimensions must be disjoint"
    has_prefix = True
    prefix_list = ["o_o_o", "j_j_j", "k_k_k"]
    for prefix in prefix_list:
        if prefix in ein_expr:
            continue
        else:
            for i, dim_set in enumerate(dims):
                new_dim_name = f"{prefix}_{i}"
                for d in dim_set:
                    ein_expr = ein_expr.replace(d, new_dim_name)
            has_prefix = False
            break

    if has_prefix:
        raise Exception(
            f"Woops, you have entered a magic dimension name, please don't use any of {prefix_list} in your ein_expr"
        )

    return einsum(*tensors, ein_expr)

In [3]:
# a is in space H1 x H2 x H3
a = torch.randn(
    2,
    2,
    2,  # a_l1, a_l2, a_l3
    2,
    2,
    2,  # a_r1, a_r2, a_r3
)

I = torch.eye(2)
# A is in space H1 x H2 x H3 x H4 by outer product of a and I
A = einsum(a, I, "al1 al2 al3 ar1 ar2 ar3, i4l i4r -> al1 al2 al3 i4l ar1 ar2 ar3 i4r")

# b is in space H3 x H4
b = torch.randn(
    2,
    2,  # b_l3, b_l4
    2,
    2,  # b_r3, b_r4
)
# B is in space H1 x H2 x H3 x H4 by outer product of two identity gates
B = einsum(I, I, b, "i1l i1r, i2l i2r, bl3 bl4 br3 br4 -> i1l i2l bl3 bl4 i1r i2r br3 br4")

# BA is in space H1 x H2 x H3 x H4 by tensor product of B and A
BA = einsum(
    B,
    A,
    "i1l i2l bl3 bl4 c1 c2 c3 c4, c1 c2 c3 c4 ar1 ar2 ar3 i4r -> i1l i2l bl3 bl4 ar1 ar2 ar3 i4r",
)
# ba is in space H1 x H2 x H3 x H4 by tensor product of b and a
ba = einsum(b, a, "bl3 bl4 c br4, al1 al2 c ar1 ar2 ar3 -> al1 al2 bl3 bl4 ar1 ar2 ar3 br4")
assert torch.allclose(BA, ba)

ba_contract = tensor_contract(
    b,
    a,
    ein_expr="bl3 bl4 br3 br4, al1 al2 al3 ar1 ar2 ar3 -> al1 al2 bl3 bl4 ar1 ar2 ar3 br4",
    dims={"br3", "al3"},
)

assert torch.allclose(ba_contract, ba)

## Product of Control Gates

TODO

## Product of Control Gate and General Gate

TODO